# Error propagation in an arbitrary multivariate function

We are going to calculate the value and propagated error of an observable $Z$ defined as

\begin{align*}
    Z = f(A, B, C, ...)
\end{align*}

For this particular example, we take 

\begin{align}
    Z(A, B, C, D) = A^2 + B \ln(C/D)
\end{align}

## Import data

The imported table contains symbols '$A$', '$B$', ..., mean values $\bar A, \bar B, ...$ and uncertainties $\alpha_A, \alpha_B$


In [193]:
import pandas as pd
import sympy as sym

parameters = pd.read_csv('parameters_table.csv').set_index('parametro')
parameters


,valor_promedio,incertidumbre
parametro,,
A,1.20,0.10
B,1.30,0.01
C,1.25,0.01
D,1.15,0.05


## define symbols and expressions

define variables as sympy symbols and $Z$ as a sympy expression

In [187]:
A, B, C, D = sym.symbols("A B C D")
Z = A**2 + B*sym.log(C/D)

In [188]:
Z_bar = Z.subs(parameters['valor_promedio'].items())
Z_bar

1.54839609162077

## partial and total errors

### partial errors 

These are calculated as the variation in $\bar Z$ with respect to each variable as follows

\begin{align*}
    \alpha_Z^A &= Z(\bar A + \alpha_A, \bar B, \bar C, \bar D) - f(\bar A, \bar B, \bar C, \bar D) \\
    &\vdots \\
    \alpha_Z^D &= Z(\bar A, \bar B, \bar C, \bar D + \alpha_D) - f(\bar A, \bar B, \bar C, \bar D)
\end{align*}

In [194]:
parameters

,valor_promedio,incertidumbre
parametro,,
A,1.20,0.10
B,1.30,0.01
C,1.25,0.01
D,1.15,0.05


For a each variable, we calculate an array containing modified arguments. For variable $A$, for instance, the array is called *alpha_A_args* and is calculated according to replacement rules

\begin{align}
    \begin{array}{c}
    A \rightarrow A + \alpha_A  \\
    \vdots \\
    D \rightarrow D
    \end{array}
\end{align}


In [ ]:

alpha_A_args = parameters['valor_promedio'].copy()
alpha_A_args.loc['A'] = alpha_A_args.loc['A'] + parameters['incertidumbre']['A']
alpha_A = Z.subs(alpha_A_args.items()) - Z_bar

For all variables

\begin{align*}
    \textit{'A'} &: 
    \begin{pmatrix}
    A \rightarrow A + \alpha_A  \\
    \vdots \\
    D \rightarrow D
    \end{pmatrix} \\
    &\qquad\qquad\vdots \\
\textit{'D'} &: 
\begin{pmatrix}
A \rightarrow A  \\
\vdots \\
D \rightarrow D + \alpha_D
\end{pmatrix}
\end{align*}

In [216]:
mean_vals = parameters['valor_promedio'].copy()
uncertainties_val = parameters['incertidumbre']
alpha_args = {}
for symbol in parameters.index:
    alpha_args[symbol] = mean_vals.copy()
    alpha_args[symbol].loc[symbol] = mean_vals.loc[symbol] + uncertainties_val.loc[symbol]
    
    

Then, the value of $(\alpha_Z^j)^2 = |Z(\text{args}_j) - \bar Z|^2$ where $\text{args}_j$ are the just calculated arrays of modified arguments. For all variables, the values are stored in the array *partial_errors*

In [225]:
partial_sq_errors = np_array([abs(Z.subs(variable.items()) - Z_bar)**2 for variable in alpha_args.values()])
total_error = partial_sq_errors.sum()
total_error


0.0656691283861597